# Openpose 1.7.0 Demo

In [15]:
# 最初に、ランタイムの種類を「GPU」に変更してください。（ランタイム＞ランタイムのタイプを変更）
# GPUが有効になっていることを確認します。

# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Jun 12 01:31:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              11W /  7

In [16]:
# Openpose バージョン指定タグ
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

v1.7.0


In [17]:
# CMakeが古いとOpenpose（CUDA10)が失敗するので、バージョンを確認します。（BugfixはCMake 3.12.3）
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
! cmake --version

cmake version 3.19.6

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [4]:
# 2021.03.03時点の最新CMakeを再ビルド（15分くらい）
# Rebuild the latest CMake as of 2021.03.03 (about 15 minutes)
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

ストリーミング出力は最後の 5000 行に切り捨てられました。
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1 -DCMake_HAVE_CXX_FILESYSTEM=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/std   -I/content/cmake-3.19.6/Utilities  -c /content/cmake-3.19.6/Source/cmTryRunCommand.cxx -o cmTryRunCommand.o
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1 -DCMake_HAVE_CXX_FILESYSTEM=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/std   -I/content/cmake-3.19.6/Utilities  -c /content/cmake-3.19.6/Source/cmUnsetCommand.cxx -o cmUnsetCommand.o
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1 -DCMake_HAVE_CXX_FILESYSTEM=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/s

In [18]:
# ライブラリインストール
# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
Reading packa

In [6]:
# Openposeのコードをclone
# Clone Openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

Cloning into 'openpose'...
remote: Enumerating objects: 659, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 659 (delta 171), reused 162 (delta 24), pack-reused 0
Receiving objects: 100% (659/659), 46.51 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (171/171), done.
Note: switching to '8ca5c1d95a42340b323e9273654d1db98bec779c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [8]:
# build用ディレクトリを作成
# Create build directory
! cd openpose && mkdir build && cd build

mkdir: cannot create directory ‘build’: File exists


In [19]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# 上記インストール手順のシナリオ１でインストール実行
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake ..

# COCOモデルのDLオプション付き(「# ! cd」の部分を「! cd」に変更してください。)
# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# MPIモデルのDLオプション付き（同上）
# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Building with CUDA.
-- CUDA detected: 12.2
-- Found cuDNN: ver. 8.9.6 found (include: /usr/include, library: /usr/lib/x86_64-linux-gnu/libcudnn.so)
-- Added CUDA NVCC flags for: sm_75
-- Found cuDNN: ver. 8.9.6 found (include: /usr/include, library: /usr/lib/x86_64-linux-gnu/libcudnn.so)
-- Found gflags  (include: /usr/include, library: /usr/lib/x86_64-linux-gnu/libgflags.so)
-- Found glog    (include: /usr/include, library: /usr/lib/x86_64-linux-gnu/libglog.so)
-- Caffe will be downloaded from source now. NOTE: This process might take several minutes depending
        on your internet connection.
-- Caffe has already been downloaded.
HEAD is now at 1807aada Added Ampere arch's (CUDA11)
-- Caffe will be built from source now.
-- Download the models.
-- Downloading BODY_25 model...
-- Model already exists.
-- Not downloading body (COCO) model
-- Not downloading body (MPI) model
-- Downloading face model...
--

In [20]:
# Openposeのビルド（15分くらい）
# Openpose Building
! cd openpose/build && make -j`nproc`
# outputフォルダ作成
! cd openpose && mkdir output

Scanning dependencies of target openpose_lib
[ 12%] Creating directories for 'openpose_lib'
[ 25%] No download step for 'openpose_lib'
[ 37%] No update step for 'openpose_lib'
[ 50%] No patch step for 'openpose_lib'
[ 62%] Performing configure step for 'openpose_lib'
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - 

In [25]:
# デモ動画を解析
# 出力された解析結果は、Colab画面の左メニューのフォルダアイコンから「openpose/output」以下に配置されます。
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --display 0  --write_video ./output/openpose.avi

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
[ WARN:0] global ./modules/core/src/matrix_expressions.cpp (1333) assign OpenCV/MatExpr: processing of multi-channel arrays might be changed in the future: https://github.com/opencv/opencv/issues/16739
OpenPose demo successfully finished. Total time: 18.990547 seconds.


In [26]:
# デモ動画の結果をffmpegで再エンコードして表示します。少し時間がかかります。
# The result of the demo video is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "./openpose/output/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from google.colab import drive
import os

# Googleドライブマウント
# マウントすると、アクセス許可が求められるので、許可して、キーをコピーして、Colab側の入力欄にCtrl+Vで貼り付けて、Enter。
# mount Google Drive
# When you mount it, you will be asked for permission, so allow it, copy the key, paste(Ctrl+V) it in the input field on the Colab side, and press Enter.
drive.mount('/gdrive')

# Google Drive のルートディレクトリ
# Google Drive root directory
root_path = "/gdrive/My Drive/"

# Googleドライブ上で解析したい動画パス（この値を好きな値に変更する事で、ご自分のGoogleドライブ上の動画を解析できます）
# Video path that you want to analyze on Google Drive (You can analyze the video on your own Google Drive by changing this value to a value you like)
file_path = "autotrace/input.mp4"

# Googleドライブ上の動画を解析（Googleドライブのルートディレクトリに解析結果の「openpose.avi」が出力されます）
# Analyze video on Google Drive (The analysis result "openpose.avi" is output to the root directory of Google Drive)
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi"

In [ ]:
# Googleドライブ上の解析結果動画をffmpegで再エンコードして表示します。少し時間がかかります。
# The analysis result video on Google Drive is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "$root_path/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

In [ ]:
# 指定した人数分のみ解析したい場合、「number_people_max」の後ろに人数を指定して下さい
# If you want to analyze only the specified number of people, please specify the number of people after "number_people_max"
# 結果はひとつ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code one level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --number_people_max 1

In [ ]:
# 手の解析も一緒に行いたい場合、「--hand」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は２つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code two level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --hand

In [ ]:
# 顔の解析も一緒に行いたい場合、「--face」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は３つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code three level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --face